In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
import datetime
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

#import csv
scores_df=pd.read_csv("../resources/merged_df.csv")
scores_df

,Unnamed: 0,Competition Title,Competition Date,FIRST,LAST,Nation,StartNr,Rank,Qualification,Semifinal,...,Quali_Atempts_Tops,Quali_Attempts_Zones,semi_Tops,semi_Zones,semi_Atempts_Tops,semi_Attempts_Zones,Finals_Tops,Finals_Zones,Final_Atempts_Tops,Final_Attempts_Zones
0,0,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Naile,MEIGNAN,FRA,15.0,1,6T7z99,3T4z55,...,9,9,3,4,5,5,3,4,8,9
1,1,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Miriam,FOGU,ITA,27.0,2,4T8z814,3T4z55,...,8,4,3,4,5,5,2,4,2,7
2,2,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Vanda,MICHALKOVA,SVK,48.0,3,6T7z89,3T3z43,...,8,9,3,3,4,3,2,3,2,3
3,3,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Lola,SAUTIER,FRA,17.0,4,4T6z69,2T3z89,...,6,9,2,3,8,9,1,3,3,8
4,4,European Youth Championships (B) - Brixen (ITA...,20 - 22 September 2019,Polina,KULAGINA,RUS,39.0,5,4T6z611,3T4z76,...,6,1,3,4,7,6,1,3,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,5427,European Youth Cup (B) - Soure (POR) 2018,5 - 6 May 2018,solene,MOREAU,FRA,44.0,13,5T8z510,NaN,...,3,6,0,0,0,0,0,0,0,0
2429,5428,European Youth Cup (B) - Soure (POR) 2018,5 - 6 May 2018,Lucie,VAILLANT BULTEL,FRA,45.0,14,5T7z99,NaN,...,4,7,0,0,0,0,0,0,0,0
2430,5429,European Youth Cup (B) - Soure (POR) 2018,5 - 6 May 2018,Helena,WALTZ,GER,81.0,15,5T7z1111,NaN,...,5,7,0,0,0,0,0,0,0,0
2431,5430,European Youth Cup (B) - Soure (POR) 2018,5 - 6 May 2018,Marie,COLOT,BEL,10.0,16,5T6z98,NaN,...,0,5,0,0,0,0,0,0,0,0


In [2]:
import pandas as pd
from sqlalchemy import create_engine
# Define the database file path
db_path = 'sqlite:///../Database/climbing.sqlite'
# Create a SQLAlchemy engine
engine = create_engine(db_path)
# Define the SQL query
query = 'SELECT * FROM climbing_table'
# Execute the SQL query and load the results into a pandas DataFrame
scores_df = pd.read_sql(query, engine)
# Print the first 5 rows of the DataFrame
print(scores_df.head())

   Unnamed: 0                                  Competition Title  \
0           0  European Youth Championships (B) - Brixen (ITA...   
1           1  European Youth Championships (B) - Brixen (ITA...   
2           2  European Youth Championships (B) - Brixen (ITA...   
3           3  European Youth Championships (B) - Brixen (ITA...   
4           4  European Youth Championships (B) - Brixen (ITA...   

          Competition Date   FIRST        LAST Nation  StartNr  Rank  \
0  20  - 22 September 2019   Naile     MEIGNAN    FRA     15.0     1   
1  20  - 22 September 2019  Miriam        FOGU    ITA     27.0     2   
2  20  - 22 September 2019   Vanda  MICHALKOVA    SVK     48.0     3   
3  20  - 22 September 2019    Lola     SAUTIER    FRA     17.0     4   
4  20  - 22 September 2019  Polina    KULAGINA    RUS     39.0     5   

  Qualification Semifinal  ... Quali_Atempts_Tops Quali_Attempts_Zones  \
0        6T7z99    3T4z55  ...                  9                    9   
1       4T

In [3]:
scores_df['competitor_id'] = pd.factorize(scores_df['FIRST'] + '_' + scores_df['LAST'])[0] + 1


In [4]:
scores_df.columns

Index(['Unnamed: 0', 'Competition Title', 'Competition Date', 'FIRST', 'LAST',
       'Nation', 'StartNr', 'Rank', 'Qualification', 'Semifinal', 'Final',
       'Category', 'Quali_Tops', 'Quali_Zones', 'Quali_Atempts_Tops',
       'Quali_Attempts_Zones', 'semi_Tops', 'semi_Zones', 'semi_Atempts_Tops',
       'semi_Attempts_Zones', 'Finals_Tops', 'Finals_Zones',
       'Final_Atempts_Tops', 'Final_Attempts_Zones', 'competitor_id'],
      dtype='object')

In [5]:
scores_df=scores_df.drop(columns=["FIRST","LAST","Qualification","Semifinal","Final","Category","Unnamed: 0",'Competition Title','semi_Tops', 'semi_Zones', 'semi_Atempts_Tops',
       'semi_Attempts_Zones', 'Finals_Tops', 'Finals_Zones',
       'Final_Atempts_Tops', 'Final_Attempts_Zones'])
scores_df

,Competition Date,Nation,StartNr,Rank,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,competitor_id
0,20 - 22 September 2019,FRA,15.0,1,6,7,9,9,1
1,20 - 22 September 2019,ITA,27.0,2,4,8,8,4,2
2,20 - 22 September 2019,SVK,48.0,3,6,7,8,9,3
3,20 - 22 September 2019,FRA,17.0,4,4,6,6,9,4
4,20 - 22 September 2019,RUS,39.0,5,4,6,6,1,5
...,...,...,...,...,...,...,...,...,...
2428,5 - 6 May 2018,FRA,44.0,13,1,2,3,6,923
2429,5 - 6 May 2018,FRA,45.0,14,1,2,4,7,924
2430,5 - 6 May 2018,GER,81.0,15,1,2,5,7,501
2431,5 - 6 May 2018,BEL,10.0,16,0,2,0,5,115


In [6]:
scores_df.nunique()

Competition Date          16
Nation                    63
StartNr                  544
Rank                      67
Quali_Tops                 9
Quali_Zones                9
Quali_Atempts_Tops        10
Quali_Attempts_Zones      22
competitor_id           1127
dtype: int64

In [7]:
scores_df["Competition Date"].value_counts()

11  - 12 May 2019               328
31 August - 2 September 2018    301
20  - 22 September 2019         282
25  - 26 May 2019               258
27  - 28 April 2019             241
2  - 3 June 2018                211
21  - 22 July 2018              193
1  - 4 November 2018            151
5  - 7 September 2019            91
7  - 11 November 2018            87
22  - 31 August 2019             59
15  - 17 June 2018               59
21  - 26 November 2018           55
9  - 16 August 2018              51
9  - 10 March 2019               49
5  - 6 May 2018                  17
Name: Competition Date, dtype: int64

In [8]:
scores_df["Nation"].value_counts()

ITA    161
FRA    150
AUT    122
BEL    114
GER    107
      ... 
KGZ      3
RSA      2
PER      2
NEP      2
MEX      1
Name: Nation, Length: 63, dtype: int64

In [9]:
encoded_df = pd.get_dummies(scores_df, columns=["Nation"])
encoded_df['competition_id'] = pd.factorize(encoded_df["Competition Date"])[0] + 1
encoded_df

,Competition Date,StartNr,Rank,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,competitor_id,Nation_ARG,Nation_AUS,...,Nation_SUI,Nation_SVK,Nation_SWE,Nation_THA,Nation_TPE,Nation_TUR,Nation_UKR,Nation_USA,Nation_VEN,competition_id
0,20 - 22 September 2019,15.0,1,6,7,9,9,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,20 - 22 September 2019,27.0,2,4,8,8,4,2,0,0,...,0,0,0,0,0,0,0,0,0,1
2,20 - 22 September 2019,48.0,3,6,7,8,9,3,0,0,...,0,1,0,0,0,0,0,0,0,1
3,20 - 22 September 2019,17.0,4,4,6,6,9,4,0,0,...,0,0,0,0,0,0,0,0,0,1
4,20 - 22 September 2019,39.0,5,4,6,6,1,5,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,5 - 6 May 2018,44.0,13,1,2,3,6,923,0,0,...,0,0,0,0,0,0,0,0,0,16
2429,5 - 6 May 2018,45.0,14,1,2,4,7,924,0,0,...,0,0,0,0,0,0,0,0,0,16
2430,5 - 6 May 2018,81.0,15,1,2,5,7,501,0,0,...,0,0,0,0,0,0,0,0,0,16
2431,5 - 6 May 2018,10.0,16,0,2,0,5,115,0,0,...,0,0,0,0,0,0,0,0,0,16


In [10]:
encoded_df.dtypes

Competition Date     object
StartNr             float64
Rank                  int64
Quali_Tops            int64
Quali_Zones           int64
                     ...   
Nation_TUR            uint8
Nation_UKR            uint8
Nation_USA            uint8
Nation_VEN            uint8
competition_id        int64
Length: 72, dtype: object

In [11]:
import re

def fix_date(date_string):
    regex_pattern = r"^(\d{1,2})\s-\s\d{1,2}\s(\w+)\s(\d{4})"
    match_obj = re.match(regex_pattern, date_string)

    if match_obj:
        output_str = f"{match_obj.group(1)} {match_obj.group(2)} {match_obj.group(3)}"
        print(output_str)
    else:
        print("No match found")
    return output_str

In [12]:
dates = encoded_df['Competition Date'].str.extractall(r"^(\d{1,2})\s*(-\s*(\d{1,2})\s+)?(\w+)\s+(\d{4})?")
dates = pd.DataFrame(dates)
dates = dates.reset_index().drop(columns=["match", "level_0",1,2]).reset_index()
dates = dates.drop(columns=["index"])
dates.columns=["day","month","year"]
dates["month"]=pd.to_datetime(dates["month"],format="%B")
dates["month"]=dates["month"].dt.month
dates = dates.fillna(9)
dates.astype({'day':'int64','year':'int64'}).dtypes
dates


,day,month,year
0,20,9,2019
1,20,9,2019
2,20,9,2019
3,20,9,2019
4,20,9,2019
...,...,...,...
2428,5,5,2018
2429,5,5,2018
2430,5,5,2018
2431,5,5,2018


In [13]:
merger_dates_df = pd.merge(encoded_df, dates, left_index=True, right_index=True)
df = merger_dates_df.drop(columns=['Competition Date'])

In [14]:
df

,StartNr,Rank,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,competitor_id,Nation_ARG,Nation_AUS,Nation_AUT,...,Nation_THA,Nation_TPE,Nation_TUR,Nation_UKR,Nation_USA,Nation_VEN,competition_id,day,month,year
0,15.0,1,6,7,9,9,1,0,0,0,...,0,0,0,0,0,0,1,20,9,2019
1,27.0,2,4,8,8,4,2,0,0,0,...,0,0,0,0,0,0,1,20,9,2019
2,48.0,3,6,7,8,9,3,0,0,0,...,0,0,0,0,0,0,1,20,9,2019
3,17.0,4,4,6,6,9,4,0,0,0,...,0,0,0,0,0,0,1,20,9,2019
4,39.0,5,4,6,6,1,5,0,0,0,...,0,0,0,0,0,0,1,20,9,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,44.0,13,1,2,3,6,923,0,0,0,...,0,0,0,0,0,0,16,5,5,2018
2429,45.0,14,1,2,4,7,924,0,0,0,...,0,0,0,0,0,0,16,5,5,2018
2430,81.0,15,1,2,5,7,501,0,0,0,...,0,0,0,0,0,0,16,5,5,2018
2431,10.0,16,0,2,0,5,115,0,0,0,...,0,0,0,0,0,0,16,5,5,2018


In [15]:
import numpy as np
 
mask = np.column_stack([encoded_df['Competition Date'].str.contains(r"March", na=False) for col in encoded_df])
encoded_df.loc[mask.any(axis=1)]

,Competition Date,StartNr,Rank,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,competitor_id,Nation_ARG,Nation_AUS,...,Nation_SUI,Nation_SVK,Nation_SWE,Nation_THA,Nation_TPE,Nation_TUR,Nation_UKR,Nation_USA,Nation_VEN,competition_id
1259,9 - 10 March 2019,201.0,1,0,4,0,0,625,0,0,...,0,0,0,0,1,0,0,0,0,7
1260,9 - 10 March 2019,214.0,2,0,3,0,4,626,0,0,...,0,0,0,0,1,0,0,0,0,7
1261,9 - 10 March 2019,222.0,3,0,3,0,6,627,0,0,...,0,0,0,1,0,0,0,0,0,7
1262,9 - 10 March 2019,215.0,4,0,2,0,3,628,0,0,...,0,0,0,0,0,0,0,0,0,7
1263,9 - 10 March 2019,219.0,5,0,1,0,2,629,0,0,...,0,0,0,0,0,0,0,0,0,7
1264,9 - 10 March 2019,208.0,6,8,8,9,8,630,0,0,...,0,0,0,0,0,0,0,0,0,7
1265,9 - 10 March 2019,211.0,7,8,8,1,10,631,0,0,...,0,0,0,0,0,0,0,0,0,7
1266,9 - 10 March 2019,216.0,8,5,6,9,7,632,0,0,...,0,0,0,0,0,0,0,0,0,7
1267,9 - 10 March 2019,207.0,9,6,6,1,7,633,0,0,...,0,0,0,0,0,0,0,0,0,7
1268,9 - 10 March 2019,202.0,10,7,8,9,9,634,0,0,...,0,0,0,0,0,0,0,0,0,7


In [16]:
import numpy as np

# Assume your rank column is named 'rank' in a pandas DataFrame named 'df'
top_n = 6  # Define the number of top ranks you want to set to 1

# Create a new column named 'rank_top' with all values initialized to 0
df['rank_top'] = np.zeros(len(df))

# Set the top 'top_n' ranks to 1 in the 'rank_top' column
df.loc[df['Rank'] <= top_n, 'rank_top'] = 1

In [17]:
df=df.drop(columns=['Rank'])
df

,StartNr,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,competitor_id,Nation_ARG,Nation_AUS,Nation_AUT,Nation_BEL,...,Nation_TPE,Nation_TUR,Nation_UKR,Nation_USA,Nation_VEN,competition_id,day,month,year,rank_top
0,15.0,6,7,9,9,1,0,0,0,0,...,0,0,0,0,0,1,20,9,2019,1.0
1,27.0,4,8,8,4,2,0,0,0,0,...,0,0,0,0,0,1,20,9,2019,1.0
2,48.0,6,7,8,9,3,0,0,0,0,...,0,0,0,0,0,1,20,9,2019,1.0
3,17.0,4,6,6,9,4,0,0,0,0,...,0,0,0,0,0,1,20,9,2019,1.0
4,39.0,4,6,6,1,5,0,0,0,0,...,0,0,0,0,0,1,20,9,2019,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,44.0,1,2,3,6,923,0,0,0,0,...,0,0,0,0,0,16,5,5,2018,0.0
2429,45.0,1,2,4,7,924,0,0,0,0,...,0,0,0,0,0,16,5,5,2018,0.0
2430,81.0,1,2,5,7,501,0,0,0,0,...,0,0,0,0,0,16,5,5,2018,0.0
2431,10.0,0,2,0,5,115,0,0,0,1,...,0,0,0,0,0,16,5,5,2018,0.0


In [18]:
df = df.dropna()
df.columns

Index(['StartNr', 'Quali_Tops', 'Quali_Zones', 'Quali_Atempts_Tops',
       'Quali_Attempts_Zones', 'competitor_id', 'Nation_ARG', 'Nation_AUS',
       'Nation_AUT', 'Nation_BEL', 'Nation_BLR', 'Nation_BRA', 'Nation_BUL',
       'Nation_CAM', 'Nation_CAN', 'Nation_CHI', 'Nation_CHN', 'Nation_CRO',
       'Nation_CZE', 'Nation_DEN', 'Nation_ECU', 'Nation_ESP', 'Nation_FIN',
       'Nation_FRA', 'Nation_GBR', 'Nation_GER', 'Nation_GRE', 'Nation_GUA',
       'Nation_HKG', 'Nation_HUN', 'Nation_INA', 'Nation_IND', 'Nation_IRI',
       'Nation_IRL', 'Nation_ISR', 'Nation_ITA', 'Nation_JPN', 'Nation_KAZ',
       'Nation_KGZ', 'Nation_KOR', 'Nation_LAT', 'Nation_LTU', 'Nation_LUX',
       'Nation_MAC', 'Nation_MAS', 'Nation_MEX', 'Nation_NED', 'Nation_NEP',
       'Nation_NOR', 'Nation_NZL', 'Nation_PER', 'Nation_PHI', 'Nation_POL',
       'Nation_POR', 'Nation_ROU', 'Nation_RSA', 'Nation_RUS', 'Nation_SGP',
       'Nation_SLO', 'Nation_SRB', 'Nation_SUI', 'Nation_SVK', 'Nation_SWE',
       '

In [19]:
encoded = pd.get_dummies(df, columns=["competition_id"])
encoded=encoded.drop(columns=["year","competitor_id","month"])
encoded

,StartNr,Quali_Tops,Quali_Zones,Quali_Atempts_Tops,Quali_Attempts_Zones,Nation_ARG,Nation_AUS,Nation_AUT,Nation_BEL,Nation_BLR,...,competition_id_7,competition_id_8,competition_id_9,competition_id_10,competition_id_11,competition_id_12,competition_id_13,competition_id_14,competition_id_15,competition_id_16
0,15.0,6,7,9,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,27.0,4,8,8,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,48.0,6,7,8,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17.0,4,6,6,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,39.0,4,6,6,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,44.0,1,2,3,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2429,45.0,1,2,4,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2430,81.0,1,2,5,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2431,10.0,0,2,0,5,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
# Split our preprocessed data into our features and target arrays
y = encoded.rank_top.values
X = encoded.drop(columns="rank_top").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

In [21]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
classifier_oversampling = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
x_res, y_res = classifier_oversampling.fit_resample(X_train, y_train)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(x_res)

# Scale the data
X_train_scaled = X_scaler.transform(x_res)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
len(X_train_scaled[0])

85

In [24]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=len(X_train_scaled[0]), activation=activation, input_dim=len(X_train_scaled[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [25]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=30,
    hyperband_iterations=2)

In [26]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_res,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 00m 08s]
val_accuracy: 0.8258528113365173

Best val_accuracy So Far: 0.8599640727043152
Total elapsed time: 00h 05m 59s
INFO:tensorflow:Oracle triggered exit


In [27]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'num_layers': 6, 'units_0': 1, 'units_1': 31, 'units_2': 46, 'units_3': 81, 'units_4': 21, 'units_5': 6, 'tuner/epochs': 30, 'tuner/initial_epoch': 10, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0068'}
{'activation': 'relu', 'num_layers': 5, 'units_0': 41, 'units_1': 1, 'units_2': 96, 'units_3': 91, 'units_4': 91, 'units_5': 11, 'tuner/epochs': 30, 'tuner/initial_epoch': 10, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0046'}
{'activation': 'relu', 'num_layers': 5, 'units_0': 41, 'units_1': 1, 'units_2': 96, 'units_3': 91, 'units_4': 91, 'units_5': 11, 'tuner/epochs': 10, 'tuner/initial_epoch': 4, 'tuner/bracket': 3, 'tuner/round': 2, 'tuner/trial_id': '0034'}


In [37]:
models = tuner.get_best_models(num_models=3)
best_model = models[0]
best_model.build()
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 85)                7310      
                                                                 
 dense_1 (Dense)             (None, 1)                 86        
                                                                 
 dense_2 (Dense)             (None, 31)                62        
                                                                 
 dense_3 (Dense)             (None, 46)                1472      
                                                                 
 dense_4 (Dense)             (None, 81)                3807      
                                                                 
 dense_5 (Dense)             (None, 21)                1722      
                                                                 
 dense_6 (Dense)             (None, 6)                 1

In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  1
hidden_nodes_layer2 = 31
hidden_nodes_layer3 = 46
hidden_nodes_layer4 = 81
hidden_nodes_layer5 = 21
hidden_nodes_layer6 = 6




nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 1)                 86        
                                                                 
 dense_8 (Dense)             (None, 31)                62        
                                                                 
 dense_9 (Dense)             (None, 31)                992       
                                                                 
 dense_10 (Dense)            (None, 46)                1472      
                                                                 
 dense_11 (Dense)            (None, 81)                3807      
                                                                 
 dense_12 (Dense)            (None, 21)                1722      
                                                                 
 dense_13 (Dense)            (None, 6)                

In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_res,epochs=100)

Epoch 1/100
89/89 [==============================] - 0s 1ms/step - loss: 0.6917 - accuracy: 0.5056
Epoch 2/100
89/89 [==============================] - 0s 1ms/step - loss: 0.6724 - accuracy: 0.5809
Epoch 3/100
89/89 [==============================] - 0s 1ms/step - loss: 0.6418 - accuracy: 0.6282
Epoch 4/100
89/89 [==============================] - 0s 1ms/step - loss: 0.6198 - accuracy: 0.6476
Epoch 5/100
89/89 [==============================] - 0s 1ms/step - loss: 0.6075 - accuracy: 0.6554
Epoch 6/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5991 - accuracy: 0.6624
Epoch 7/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5940 - accuracy: 0.6663
Epoch 8/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5894 - accuracy: 0.6706
Epoch 9/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5855 - accuracy: 0.6702
Epoch 10/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5796 - accuracy: 0.6691
Epoch 11/

89/89 [==============================] - 0s 1ms/step - loss: 0.5526 - accuracy: 0.6748
Epoch 84/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5541 - accuracy: 0.6723
Epoch 85/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5536 - accuracy: 0.6720
Epoch 86/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5548 - accuracy: 0.6720
Epoch 87/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5558 - accuracy: 0.6688
Epoch 88/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5539 - accuracy: 0.6723
Epoch 89/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5536 - accuracy: 0.6720
Epoch 90/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5532 - accuracy: 0.6720
Epoch 91/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5531 - accuracy: 0.6702
Epoch 92/100
89/89 [==============================] - 0s 1ms/step - loss: 0.5574 - accuracy: 0.6709
Epoch 93/100


In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

18/18 - 0s - loss: 0.6102 - accuracy: 0.4632 - 153ms/epoch - 9ms/step
Loss: 0.6101572513580322, Accuracy: 0.46319568157196045


In [42]:
# Making predictions using the testing data
predictions = nn.predict(X_test_scaled)

18/18 [==============================] - 0s 941us/step


In [43]:
predictions
predictions = np.round(predictions, 0)
predictions

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],

In [44]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [45]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,187,286
Actual 1,13,71


Accuracy Score : 0.4631956912028725
Classification Report
              precision    recall  f1-score   support

         0.0       0.94      0.40      0.56       473
         1.0       0.20      0.85      0.32        84

    accuracy                           0.46       557
   macro avg       0.57      0.62      0.44       557
weighted avg       0.82      0.46      0.52       557



In [46]:
nn.save('ClimberNeuralNet3.h5')